In [ ]:
# Importamos la librería 'drive' de Google Colab, que nos permitirá montar nuestro Google Drive
from google.colab import drive

# Montamos Google Drive en el entorno de Google Colab.
# Esto permite acceder a los archivos almacenados en tu Google Drive desde el entorno de Colab.
# '/content/drive' es el punto de montaje, donde podrás ver tus archivos.
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Importamos la librería 'glob', que permite realizar búsquedas de archivos que coincidan con un patrón especificado.
import glob

# Definimos la ruta base donde se encuentran los archivos JSON.
# Esta ruta hace uso de '**', lo que permite buscar archivos en subcarpetas de forma recursiva.
# 'recursive=True' se asegura de que se busque en todas las subcarpetas dentro de la ruta especificada.
ruta_base = "/content/drive/MyDrive/Dataton_Anticorrupcion/PDN_S1/PDN_S1/**/*.json"

# Usamos glob.glob para obtener todos los archivos que coinciden con el patrón dado
archivos = glob.glob(ruta_base, recursive=True)

# Calculamos el total de archivos encontrados, que es simplemente la longitud de la lista 'archivos'
total_archivos = len(archivos)

# Mostramos el total de archivos encontrados
total_archivos

16047

In [ ]:
# Importamos la librería 'random', que nos permite realizar operaciones
# aleatorias, como seleccionar una muestra aleatoria de una lista.
import random

# Definimos el porcentaje que queremos de la muestra (10% en este caso)
porcentaje = 0.10

# Calculamos el tamaño de la muestra. Usamos max() para asegurar que el tamaño mínimo de la muestra sea 1
tam = max(1, int(total_archivos * porcentaje))

# Establecemos una semilla fija (42) para hacer que los resultados sean reproducibles.
# Esto significa que cada vez que se ejecute el código con la misma semilla,
# la muestra seleccionada será la misma.
random.seed(42)

# Seleccionamos una muestra aleatoria de archivos de tamaño 'tam'
muestra = random.sample(archivos, tam)

# Mostramos la longitud de la muestra seleccionada y el tamaño calculado de la muestra
len(muestra), tam

(1604, 1604)

In [ ]:
# Importamos las librerías necesarias
import json
import numpy as np

# Función que extrae el valor de 'valor' de un diccionario si existe,
# si no, regresa NaN (valor no disponible)
def valor(obj):
    """Extrae obj['valor'] si existe, si no regresa NaN."""
    if isinstance(obj, dict):
        return obj.get("valor", np.nan)
    return np.nan

# Función que extrae el valor de 'remuneracionTotal' -> 'valor' si existe
def remuneracion(obj):
    """Extrae remuneracionTotal['valor'] si existe."""
    if isinstance(obj, dict):
        r = obj.get("remuneracionTotal")
        if isinstance(r, dict):
            return r.get("valor", np.nan)
    return np.nan

# Función que suma el valor de los bienes donde el titular es 'DEC'
def sumar_bienes(lista):
    """Suma bienes donde el titular es DEC."""
    if not isinstance(lista, list):
        return np.nan
    total = 0
    for b in lista:
        titulares = b.get("titular", [])
        # Verifica si alguno de los titulares tiene la clave "DEC"
        es_dec = any(t.get("clave") == "DEC" for t in titulares if isinstance(t, dict))
        if not es_dec:
            continue
        # Extrae el valor de adquisición del bien
        v = b.get("valorAdquisicion", {}).get("valor")
        if isinstance(v, (int, float)):
            total += v
    return total if total > 0 else np.nan

# Función que suma los adeudos del declarante
def sumar_adeudos(lista):
    """Suma adeudos del declarante."""
    if not isinstance(lista, list):
        return np.nan
    total = 0
    for a in lista:
        titulares = a.get("titular", [])
        # Verifica si alguno de los titulares tiene la clave "DEC"
        es_dec = any(t.get("clave") == "DEC" for t in titulares if isinstance(t, dict))
        if not es_dec:
            continue
        # Extrae el monto del adeudo
        v = a.get("montoOriginal", {}).get("valor")
        if isinstance(v, (int, float)):
            total += v
    return total if total > 0 else np.nan

# -----------------------------------------
# LECTURA OPTIMIZADA DE ARCHIVOS
# -----------------------------------------
filas = []  # Lista para almacenar los resultados

# Iteramos sobre cada ruta de archivo en la muestra
for ruta in muestra:
    try:
        # Abrimos el archivo en modo lectura y cargamos su contenido JSON
        with open(ruta, "r", encoding="utf-8") as f:
            data = json.load(f)

        # Iteramos sobre cada declaración dentro del archivo (cada archivo tiene varias declaraciones)
        for d in data:

            # Creamos un diccionario con las columnas necesarias (compactas) para la fila
            fila = {
                # Datos generales de la declaración
                "id": d.get("id"),
                "anio": d.get("anioEjercicio"),
                "tipo": d.get("metadata", {}).get("tipo"),
                "institucion": d.get("metadata", {}).get("institucion"),

                # Identidad del declarante
                "nombre": d.get("declaracion", {}).get("situacionPatrimonial", {}).get("datosGenerales", {}).get("nombre"),
                "primerApellido": d.get("declaracion", {}).get("situacionPatrimonial", {}).get("datosGenerales", {}).get("primerApellido"),
                "segundoApellido": d.get("declaracion", {}).get("situacionPatrimonial", {}).get("datosGenerales", {}).get("segundoApellido"),

                # Información sobre el puesto que ocupa el declarante
                "cargo": d.get("declaracion", {}).get("situacionPatrimonial", {}).get("datosEmpleoCargoComision", {}).get("empleoCargoComision"),
                "nivelGobierno": d.get("declaracion", {}).get("situacionPatrimonial", {}).get("datosEmpleoCargoComision", {}).get("nivelOrdenGobierno"),
                "ente": d.get("declaracion", {}).get("situacionPatrimonial", {}).get("datosEmpleoCargoComision", {}).get("nombreEntePublico"),

                # Ingresos del declarante
                "ingreso_cargo": valor(d.get("declaracion", {}).get("situacionPatrimonial", {}).get("ingresos", {}).get("remuneracionAnualCargoPublico")),
                "ingreso_neto": valor(d.get("declaracion", {}).get("situacionPatrimonial", {}).get("ingresos", {}).get("ingresoAnualNetoDeclarante")),
                "ingreso_industrial": remuneracion(d.get("declaracion", {}).get("situacionPatrimonial", {}).get("ingresos", {}).get("actividadIndustrialComercialEmpresarial")),
                "ingreso_financiero": remuneracion(d.get("declaracion", {}).get("situacionPatrimonial", {}).get("ingresos", {}).get("actividadFinanciera")),
                "ingreso_profesional": remuneracion(d.get("declaracion", {}).get("situacionPatrimonial", {}).get("ingresos", {}).get("serviciosProfesionales")),
                "ingreso_enajenacion": remuneracion(d.get("declaracion", {}).get("situacionPatrimonial", {}).get("ingresos", {}).get("enajenacionBienes")),
                "otros_ingresos": remuneracion(d.get("declaracion", {}).get("situacionPatrimonial", {}).get("ingresos", {}).get("otrosIngresos")),

                # Patrimonio del declarante
                "inmuebles_total": sumar_bienes(d.get("declaracion", {}).get("situacionPatrimonial", {}).get("bienesInmuebles", {}).get("bienInmueble")),
                "vehiculos_total": sumar_bienes(d.get("declaracion", {}).get("situacionPatrimonial", {}).get("vehiculos", {}).get("vehiculo")),
                "muebles_total": sumar_bienes(d.get("declaracion", {}).get("situacionPatrimonial", {}).get("bienesMuebles", {}).get("bienMueble")),

                # Adeudos del declarante
                "adeudos_total": sumar_adeudos(d.get("declaracion", {}).get("situacionPatrimonial", {}).get("adeudos", {}).get("adeudo")),
            }

            # Agregamos la fila a la lista de filas
            filas.append(fila)

    except Exception as e:
        # Si ocurre un error durante la lectura del archivo, lo reportamos
        print("Error:", ruta, e)

# Finalmente, mostramos la cantidad de filas procesadas
len(filas)

695813

In [ ]:
import pandas as pd

# Convertimos la lista en un DataFrame de pandas.
df = pd.DataFrame(filas)

# Mostramos las primeras 5 filas del DataFrame para revisar cómo se estructuraron los datos.
df.head()

,id,anio,tipo,institucion,nombre,primerApellido,segundoApellido,cargo,nivelGobierno,ente,...,ingreso_neto,ingreso_industrial,ingreso_financiero,ingreso_profesional,ingreso_enajenacion,otros_ingresos,inmuebles_total,vehiculos_total,muebles_total,adeudos_total
0,UARP630202U481,NaN,INICIAL,"INSTITUTO NACIONAL DE TRANSPARENCIA, ACCESO A ...",PORFIRIO,UGALDE,RESÉNDIZ,Subdirección de Área,FEDERAL,"INSTITUTO NACIONAL DE TRANSPARENCIA, ACCESO A ...",...,NaN,NaN,0.0,0.0,NaN,22235.0,1783868.0,NaN,117000.0,208500.0
1,21le12ffgp,NaN,INICIAL,CUITZEO,J. GUADALUPE,RODRIGUEZ,VALDES,ELEMENTO,MUNICIPAL_ALCALDIA,CUITZEO,...,NaN,0.0,0.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN
2,177346s,NaN,MODIFICACIÓN,Secretaría de la Función Pública del Estado de...,MARIA ISABEL,RODRIGUEZ,MURILLO,AUXILIAR ADMINISTRATIVO,ESTATAL,Sistema Estatal para el Desarrollo Integral de...,...,122319.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,80161s,NaN,MODIFICACIÓN,Secretaría de la Función Pública del Estado de...,MARIA ISABEL,AMADOR,PEREZ,SUB OFICIAL,ESTATAL,Secretaría de Seguridad Pública,...,212766.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,179797s,NaN,MODIFICACIÓN,Secretaría de la Función Pública del Estado de...,MARIA ISABEL,AMADOR,PEREZ,SUB OFICIAL,ESTATAL,Secretaría de Seguridad Pública,...,233370.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Verificamos los tipos de datos de cada columna en el DataFrame.
df.dtypes

,0
id,object
anio,float64
tipo,object
institucion,object
nombre,object
primerApellido,object
segundoApellido,object
cargo,object
nivelGobierno,object
ente,object


In [ ]:
# Seleccionamos las columnas numéricas (enteros y flotantes)
num_cols = df.select_dtypes(include=["int64", "float64"]).columns.tolist()

# Seleccionamos las columnas categóricas (objetos o cadenas de texto)
cat_cols = df.select_dtypes(include=["object"]).columns.tolist()

# Mostramos las listas de columnas numéricas y categóricas
num_cols, cat_cols

(['anio',
  'ingreso_cargo',
  'ingreso_neto',
  'ingreso_industrial',
  'ingreso_financiero',
  'ingreso_profesional',
  'ingreso_enajenacion',
  'otros_ingresos',
  'inmuebles_total',
  'vehiculos_total',
  'muebles_total',
  'adeudos_total'],
 ['id',
  'tipo',
  'institucion',
  'nombre',
  'primerApellido',
  'segundoApellido',
  'cargo',
  'nivelGobierno',
  'ente'])

In [ ]:
# Calculamos el número total de valores nulos por columna
nulos_abs = df.isna().sum()

# Calculamos el porcentaje de valores nulos por columna
nulos_rel = df.isna().mean() * 100  # porcentaje

# Creamos un DataFrame resumen con los totales y porcentajes de nulos
resumen_nulos = pd.DataFrame({
    "nulos": nulos_abs,
    "porcentaje_nulos": nulos_rel.round(2)
}).sort_values("porcentaje_nulos", ascending=False)  # Ordenamos por porcentaje de nulos

# Mostramos el resumen de los nulos
resumen_nulos

,nulos,porcentaje_nulos
anio,693813,99.71
adeudos_total,686350,98.64
muebles_total,679293,97.63
inmuebles_total,666813,95.83
vehiculos_total,653989,93.99
ingreso_industrial,525563,75.53
ingreso_enajenacion,144363,20.75
ingreso_neto,129854,18.66
ingreso_cargo,129853,18.66
ingreso_profesional,16120,2.32


# Modelo

In [ ]:
import numpy as np
import pandas as pd

# Copia de trabajo
data = df.copy()

# 1. ASEGURAR Y LIMPIAR COLUMNAS NUMÉRICAS

In [ ]:
columnas_ingresos = [
    'ingreso_cargo', 'ingreso_industrial', 'ingreso_financiero',
    'ingreso_profesional', 'ingreso_enajenacion', 'otros_ingresos'
]  # Lista de columnas relacionadas con distintos tipos de ingresos

columnas_patrimonio = [
    'inmuebles_total', 'vehiculos_total', 'muebles_total'
]  # Lista de columnas relacionadas con el patrimonio declarado

columnas_numericas = columnas_ingresos + columnas_patrimonio + ['adeudos_total']
# Unimos ingresos, patrimonio y adeudos en una sola lista de columnas numéricas

# Asegurar que las columnas existan en el DataFrame y que sean numéricas
for col in columnas_numericas:
    if col not in data.columns:
        data[col] = 0  # Si la columna no existe, se crea con valor 0
    data[col] = pd.to_numeric(data[col], errors="coerce").fillna(0)
    # Convertimos la columna a numérico; valores no convertibles se ponen como NaN y luego se reemplazan por 0

# 2. CALCULAR TOTAL DE INGRESOS

In [ ]:
data['total_ingresos'] = (
    data['ingreso_cargo']
    + data['ingreso_industrial']
    + data['ingreso_financiero']
    + data['ingreso_profesional']
    + data['ingreso_enajenacion']
    + data['otros_ingresos']
    # Sumamos todos los tipos de ingreso declarados para obtener el ingreso total
).fillna(0)  # Si el resultado es NaN (por valores faltantes), lo reemplazamos por 0

# 3. CALCULAR PATRIMONIO BRUTO

In [ ]:
data['patrimonio_bruto'] = (
    data['inmuebles_total']
    + data['vehiculos_total']
    + data['muebles_total']
    # Sumamos el valor de inmuebles, vehículos y muebles para obtener el patrimonio bruto
).fillna(0)  # Cualquier resultado NaN se reemplaza por 0

# 4. CALCULAR PROPORCIÓN DE OTROS INGRESOS

In [ ]:
data['prop_otros_ingresos'] = data.apply(
    lambda x: x['otros_ingresos'] / x['total_ingresos']
    if x['total_ingresos'] > 0 else 0,  # Si no hay ingresos totales, evitamos división entre 0 y asignamos 0
    axis=1  # Aplicamos la función fila por fila
)
# prop_otros_ingresos: proporción que representan "otros_ingresos" respecto al total de ingresos

In [ ]:
# 5. REGLAS R1–R10

In [ ]:
# R1 – Otros ingresos moderados (10% a 30%)
data['R1_otros_ingresos_moderados'] = (
    (data['prop_otros_ingresos'] >= 0.10) &
    (data['prop_otros_ingresos'] < 0.30)
).astype(int)

# R2 – Inconsistencia menor: ingreso pero patrimonio cero
data['R2_inconsistencia_menor'] = (
    (data['ingreso_cargo'] > 0) &
    (data['patrimonio_bruto'] == 0)
).astype(int)

# R3 – Otros ingresos altos (>= 50%)
data['R3_otros_ingresos_alto'] = (
    data['prop_otros_ingresos'] >= 0.50
).astype(int)

# R4 – Alto ingreso con patrimonio cero (> P90)
p90_ingreso = data['total_ingresos'].quantile(0.90)
data['R4_alto_ingreso_sin_patrimonio'] = (
    (data['total_ingresos'] >= p90_ingreso) &
    (data['patrimonio_bruto'] == 0)
).astype(int)

# R5 – Inconsistencia grave: patrimonio negativo
data['R5_inconsistencia_grave'] = (
    data['patrimonio_bruto'] < 0
).astype(int)

# R6 – Patrimonio fragmentado: tiene todos los tipos + adeudos
data['R6_patrimonio_fragmentado'] = (
    (data['inmuebles_total'] > 0) &
    (data['vehiculos_total'] > 0) &
    (data['muebles_total'] > 0) &
    (data['adeudos_total'] > 0)
).astype(int)

# R7 – Solo pasivos
data['R7_solo_pasivos'] = (
    (data['patrimonio_bruto'] == 0) &
    (data['adeudos_total'] > 0)
).astype(int)

# R8 – Rendimientos imposibles: ingreso 0 pero tiene patrimonio alto
data['R8_rendimientos_imposibles'] = (
    (data['total_ingresos'] == 0) &
    (data['patrimonio_bruto'] > 100000)
).astype(int)

# R9 – Outlier extremo: total_ingresos > P99
p99_ingreso = data['total_ingresos'].quantile(0.99)
data['R9_outlier_extremo'] = (
    data['total_ingresos'] >= p99_ingreso
).astype(int)

# R10 – Ratio anormal: deuda > patrimonio * 3
data['R10_ratio_anormal'] = (
    data['adeudos_total'] > (data['patrimonio_bruto'] * 3)
).astype(int)

# 6. SCORE REGLAS

In [ ]:
lista_reglas = [
    'R1_otros_ingresos_moderados',
    'R2_inconsistencia_menor',
    'R3_otros_ingresos_alto',
    'R4_alto_ingreso_sin_patrimonio',
    'R5_inconsistencia_grave',
    'R6_patrimonio_fragmentado',
    'R7_solo_pasivos',
    'R8_rendimientos_imposibles',
    'R9_outlier_extremo',
    'R10_ratio_anormal'
]
# Lista de columnas binarias (0/1) que indican si cada regla de riesgo se activó para la declaración

data['score_reglas'] = data[lista_reglas].sum(axis=1)
# Sumamos cuántas reglas se activan por declaración.
# 'score_reglas' representa el número total de banderas/reglas de riesgo cumplidas por cada registro.

# 7. ISOLATION FOREST (ANOMALÍAS)

In [ ]:
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
# Importamos las clases necesarias:
# - IsolationForest: modelo para detectar anomalías
# - StandardScaler: estandariza las variables
# - SimpleImputer: rellena valores faltantes
# - Pipeline: encadena los pasos de preprocesamiento y modelo

features = [
    'total_ingresos',
    'patrimonio_bruto',
    'ingreso_cargo',
    'otros_ingresos',
    'prop_otros_ingresos',
    'inmuebles_total',
    'vehiculos_total',
    'muebles_total',
    'adeudos_total',
    'score_reglas'
]
# Lista de variables numéricas que usaremos como entrada del modelo de anomalías

# Asegurar que todas las columnas existan
for f in features:
    if f not in data.columns:
        data[f] = 0  # Si falta alguna columna, se crea con valor 0

X = data[features].copy()
# Creamos una copia del subconjunto de datos que usará el modelo

iso_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    # Primer paso: imputar valores faltantes usando la mediana de cada columna
    ('scaler', StandardScaler()),
    # Segundo paso: escalar las variables para que tengan media 0 y varianza 1
    ('model', IsolationForest(
        n_estimators=300,      # Número de árboles en el bosque
        contamination=0.05,    # Porcentaje aproximado de observaciones anómalas
        random_state=42        # Semilla para reproducibilidad
    ))
])
# Definimos un pipeline que encadena imputación, escalado y el modelo de Isolation Forest

iso_pipeline.fit(X)
# Entrenamos el pipeline completo con los datos X

raw_scores = -iso_pipeline.named_steps['model'].decision_function(X)
# Obtenemos los "scores" de anomalía del Isolation Forest.
# Se multiplican por -1 porque por defecto valores más bajos son más anómalos;
# así los invertimos para que valores más altos = mayor riesgo.

# Normalización 0–100
min_s, max_s = raw_scores.min(), raw_scores.max()
riesgo_norm = 100 * (raw_scores - min_s) / (max_s - min_s)
# Escalamos los scores a una escala de 0 a 100 para interpretarlos como un índice de riesgo

data['riesgo_modelo'] = riesgo_norm.round(2)
# Guardamos el índice de riesgo normalizado en el DataFrame, redondeado a 2 decimales

# Etiqueta de anomalía
data['anomaly_iforest'] = (data['riesgo_modelo'] >= 80).astype(int)
# Creamos una bandera binaria: 1 si el riesgo es alto (>= 80), 0 en caso contrario

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but IsolationForest was fitted without feature names
  warnings.warn(


# 8. EXPORTAR A GOOGLE DRIVE

In [ ]:
ruta_drive = "/content/drive/MyDrive/Dataton_Anticorrupcion/resultados_anticorrupcion_final.csv"
data.to_csv(ruta_drive, index=False)

print("CSV guardado correctamente en:", ruta_drive)

data.head()

CSV guardado correctamente en: /content/drive/MyDrive/Dataton_Anticorrupcion/resultados_anticorrupcion_final.csv


,id,anio,tipo,institucion,nombre,primerApellido,segundoApellido,cargo,nivelGobierno,ente,...,R4_alto_ingreso_sin_patrimonio,R5_inconsistencia_grave,R6_patrimonio_fragmentado,R7_solo_pasivos,R8_rendimientos_imposibles,R9_outlier_extremo,R10_ratio_anormal,score_reglas,riesgo_modelo,anomaly_iforest
0,UARP630202U481,NaN,INICIAL,"INSTITUTO NACIONAL DE TRANSPARENCIA, ACCESO A ...",PORFIRIO,UGALDE,RESÉNDIZ,Subdirección de Área,FEDERAL,"INSTITUTO NACIONAL DE TRANSPARENCIA, ACCESO A ...",...,0,0,0,0,0,0,0,1,96.64,1
1,21le12ffgp,NaN,INICIAL,CUITZEO,J. GUADALUPE,RODRIGUEZ,VALDES,ELEMENTO,MUNICIPAL_ALCALDIA,CUITZEO,...,0,0,0,0,0,0,0,0,47.45,0
2,177346s,NaN,MODIFICACIÓN,Secretaría de la Función Pública del Estado de...,MARIA ISABEL,RODRIGUEZ,MURILLO,AUXILIAR ADMINISTRATIVO,ESTATAL,Sistema Estatal para el Desarrollo Integral de...,...,0,0,0,0,0,0,0,1,48.63,0
3,80161s,NaN,MODIFICACIÓN,Secretaría de la Función Pública del Estado de...,MARIA ISABEL,AMADOR,PEREZ,SUB OFICIAL,ESTATAL,Secretaría de Seguridad Pública,...,0,0,0,0,0,0,0,1,48.63,0
4,179797s,NaN,MODIFICACIÓN,Secretaría de la Función Pública del Estado de...,MARIA ISABEL,AMADOR,PEREZ,SUB OFICIAL,ESTATAL,Secretaría de Seguridad Pública,...,0,0,0,0,0,0,0,1,48.63,0


# 9 Metadatos JSON

In [ ]:
from datetime import datetime
import json
import pandas as pd # Import pandas if not already imported in this cell

# Total de declaraciones analizadas (usamos el DataFrame final: data)
total_declaraciones = int(len(data))

# Categorizar 'riesgo_modelo' en niveles de riesgo
bins = [0, 50, 80, 101] # Definimos los rangos para los niveles de riesgo
labels = ['Bajo', 'Medio', 'Alto'] # Etiquetas para cada nivel
data['riesgo_nivel'] = pd.cut(data['riesgo_modelo'], bins=bins, labels=labels, right=False, include_lowest=True)

# Distribución absoluta de los niveles de riesgo (cuántas declaraciones hay en cada nivel)
dist_abs = data['riesgo_nivel'].value_counts().to_dict()

# Distribución porcentual de los niveles de riesgo (proporción por nivel, redondeada a 3 decimales)
dist_pct = data['riesgo_nivel'].value_counts(normalize=True).round(3).to_dict()

# Diccionario con los metadatos del análisis y del modelo de riesgo
metadata = {
    "fecha_analisis": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),   # Fecha y hora de generación
    "total_declaraciones": total_declaraciones,                       # Número total de registros en 'data'
    "porcentaje_muestra": 10,                                         # Ajusta si usaste otro % de muestra
    "version_modelo": "v1.0_iso_rules",                               # Versión de tu modelo
    "descripcion_modelo": "Score de riesgo con reglas + IsolationForest",
    "columnas_entrada_modelo": features,                              # Lista de columnas usadas en el modelo
    "distribucion_riesgo_absoluta": dist_abs,                         # Conteos por nivel de riesgo
    "distribucion_riesgo_porcentual": dist_pct                        # Porcentajes por nivel de riesgo
}

# Guardar los metadatos en un archivo JSON
with open('metadata_riesgo.json', 'w', encoding='utf-8') as f:
    json.dump(metadata, f, ensure_ascii=False, indent=2)  # indent=2 para que sea legible

print("JSON generado: metadata_riesgo.json")

metadata  # Mostrar el diccionario en la salida del notebook

JSON generado: metadata_riesgo.json


{'fecha_analisis': '2025-12-01 04:52:42',
 'total_declaraciones': 695813,
 'porcentaje_muestra': 10,
 'version_modelo': 'v1.0_iso_rules',
 'descripcion_modelo': 'Score de riesgo con reglas + IsolationForest',
 'columnas_entrada_modelo': ['total_ingresos',
  'patrimonio_bruto',
  'ingreso_cargo',
  'otros_ingresos',
  'prop_otros_ingresos',
  'inmuebles_total',
  'vehiculos_total',
  'muebles_total',
  'adeudos_total',
  'score_reglas'],
 'distribucion_riesgo_absoluta': {'Bajo': 619667,
  'Medio': 71440,
  'Alto': 4706},
 'distribucion_riesgo_porcentual': {'Bajo': 0.891,
  'Medio': 0.103,
  'Alto': 0.007}}